In [69]:
# Dependencies

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
import warnings
warnings.filterwarnings("ignore")

In [70]:
# Data preprocessing

data_raw = pd.read_csv('activity_sample_data.csv',nrows=900,index_col='id')
data_raw = data_raw[['activitytypeid','exchangerate','totalactivitycost','planid','activityrejectionreasonid']]
data_raw = data_raw.fillna(0)
data_raw['rejection_decision'] = [0 if x != 0 else 1 for x in data_raw['activityrejectionreasonid']] #1 indicates decision was accepted
data_raw = pd.get_dummies(data_raw)
X_regression = data_raw[[x for x in data_raw.columns if x not in ['totalactivitycost','rejection_decision']]]
y_regression = data_raw['totalactivitycost']
X_classification = data_raw[[x for x in data_raw.columns if x not in ['totalactivitycost','rejection_decision']]]
y_classification = data_raw['rejection_decision']

In [71]:
# Train test split for regression algorithms

reg_train, reg_test, regy_train, regy_test = train_test_split(X_regression, y_regression, test_size=0.2, random_state=42)

In [72]:
# Train test split for classification algorithms

class_train, class_test, classy_train, classy_test = train_test_split(X_classification, y_classification, test_size=0.2, random_state=42)

In [73]:
# Binary Classification Model - Gradient Boosted Tree

X_train = class_train.as_matrix()
X_test = class_test.as_matrix()
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xgb_model.fit(X_train,classy_train)
y_pred = xgb_model.predict(X_test)
print(confusion_matrix(classy_test, y_pred))

[[  4   0]
 [  0 176]]


In [74]:
# OLS multiple Regression

reg = LinearRegression().fit(np.array(reg_train), np.array(regy_train))
predictions = reg.predict(reg_test)

# More complicated Regression model

X_train = reg_train.as_matrix()
X_test = reg_test.as_matrix()
xgb_model = xgb.XGBRegressor(objective="reg:linear", random_state=42)
xgb_model.fit(X_train, regy_train)
y_pred = xgb_model.predict(X_test)
mse=mean_squared_error(regy_test, y_pred)
print(np.sqrt(mse))

279222.14071035694
